# Assignment - Implementing graph algorithms with LLMs

## 1 - Introduction

Welcome to the first assignment of Course 1 in the Generative AI for Software Development Specialization! In this assignment you will **work alongside an LLM to take on four different graph-based problems** including finding the shortest path through a graph and solving different variations of the classic Travelling Salesman Problem in which you must find the shortest tour through every vertex in the graph. This is a great opportunity to practice using the LLM skills you've been learning! You can use the LLM of your choosing but [GPT-4o is available here](https://www.coursera.org/learn/introduction-to-generative-ai-for-software-development/ungradedLab/Vuqvf/gpt-3-5-environment), in the ungraded lab that accompanies this assignment.

**Here's a quick summary of the contents of this notebook:**

- Section 1: Introduction, setup, and the `Graph` class this assignment is based on
- Section 2: The `Graph_Advanced` class that you will edit for your submission
- Section 3: Description of the four exercises you'll implement inside of `Graph_Advanced`
- Section 4: A playground to test solutions
- Section 5: Unit tests of your `Graph_Advanced` class

**Here's a few important points on how this notebook works:**

- **Only `Graph_Advanced` is Graded:** You can add new cells to experiment but these will be ignored by the grader. If you want something graded, include it in the provided cell that contains the `Graph_Advanced` class.
- **Some Cells are Frozen:** Some cells are frozen, e.g. the `Graph` class, to avoid mistakenly editing their code
- **Avoid importing new libraries:** Avoid importing new libraries beyond those you'll find in section 1.1, immediately below. In particular `joblib` or any multiprocessing or parallel computing approach will crash the grader. If you absolutely need additional libraries, import them in the same cell as the `Graph_Advanced` class.
- **Save before submitting:** Do this to ensure you are graded on your most recent work.

Happy coding!

### 1.1 - Importing necessary libraries

The code below imports libraries used in this assignment. You should be able to complete this assignment without additional libraries. If you absolutely need additional libraries, import them in the same cell as the `Graph_Advanced` class.

In [66]:
import random
import heapq
import os
import numpy as np
import itertools

### 1.2 - Importing unittests

This library includes unit tests to evaluate your solutions. More information on unit testing is in Section 5.

In [67]:
import unittests

### 1.3 - The `Graph` Class

The `Graph` class that defines the structure of a graph and the methods that can be performed on them. Take a moment to familiarize yourself with the class's structure and its methods, or better yet, share the code with an LLM and ask it for an explanation!

The cell below is frozen and you cannot edit the class. Instead, you will work inside of the `Graph_Advanced` class which inherits from this one.

In [68]:
class Graph:
    def __init__(self, directed=False):
        """
        Initialize the Graph.

        Parameters:
        - directed (bool): Specifies whether the graph is directed. Default is False (undirected).

        Attributes:
        - graph (dict): A dictionary to store vertices and their adjacent vertices (with weights).
        - directed (bool): Indicates whether the graph is directed.
        """
        self.graph = {}
        self.directed = directed
    
    def add_vertex(self, vertex):
        """
        Add a vertex to the graph.

        Parameters:
        - vertex: The vertex to add. It must be hashable.

        Ensures that each vertex is represented in the graph dictionary as a key with an empty dictionary as its value.
        """
        if not isinstance(vertex, (int, str, tuple)):
            raise ValueError("Vertex must be a hashable type.")
        if vertex not in self.graph:
            self.graph[vertex] = {}
    
    def add_edge(self, src, dest, weight):
        """
        Add a weighted edge from src to dest. If the graph is undirected, also add from dest to src.

        Parameters:
        - src: The source vertex.
        - dest: The destination vertex.
        - weight: The weight of the edge.
        
        Prevents adding duplicate edges and ensures both vertices exist.
        """
        if src not in self.graph or dest not in self.graph:
            raise KeyError("Both vertices must exist in the graph.")
        if dest not in self.graph[src]:  # Check to prevent duplicate edges
            self.graph[src][dest] = weight
        if not self.directed and src not in self.graph[dest]:
            self.graph[dest][src] = weight
    
    def remove_edge(self, src, dest):
        """
        Remove an edge from src to dest. If the graph is undirected, also remove from dest to src.

        Parameters:
        - src: The source vertex.
        - dest: The destination vertex.
        """
        if src in self.graph and dest in self.graph[src]:
            del self.graph[src][dest]
        if not self.directed:
            if dest in self.graph and src in self.graph[dest]:
                del self.graph[dest][src]
    
    def remove_vertex(self, vertex):
        """
        Remove a vertex and all edges connected to it.

        Parameters:
        - vertex: The vertex to be removed.
        """
        if vertex in self.graph:
            # Remove any edges from other vertices to this one
            for adj in list(self.graph):
                if vertex in self.graph[adj]:
                    del self.graph[adj][vertex]
            # Remove the vertex entry itself
            del self.graph[vertex]
    
    def get_adjacent_vertices(self, vertex):
        """
        Get a list of vertices adjacent to the specified vertex.

        Parameters:
        - vertex: The vertex whose neighbors are to be retrieved.

        Returns:
        - List of adjacent vertices. Returns an empty list if vertex is not found.
        """
        return list(self.graph.get(vertex, {}).keys())

    def _get_edge_weight(self, src, dest):
        """
        Get the weight of the edge from src to dest.

        Parameters:
        - src: The source vertex.
        - dest: The destination vertex.

        Returns:
        - The weight of the edge. If the edge does not exist, returns infinity.
        """
        return self.graph[src].get(dest, float('inf'))
    
    def __str__(self):
        """
        Provide a string representation of the graph's adjacency list for easy printing and debugging.

        Returns:
        - A string representation of the graph dictionary.
        """
        return str(self.graph)

## 2 - The `Graph_Advanced` Class

The `Graph_Advanced` class inherits from the `Graph` class above. Your submission for this assignment will be an updated version of this class which implements 4 methods, each corresponding to an exercise described in Section 3. **The cell below is the only one graded in this assignment and the primary place you will develop your solution.**

**For your methods to be graded properly, you can't change the names of the four provided methods or the structure of their parameters or returns.** You can, however, add new helper methods within the `Graph_Advanced` class that are called by the four that are provided.

In [69]:
import heapq
from itertools import combinations, permutations
import itertools
import random
import time

class Graph_Advanced(Graph):
    
    def shortest_path(self, start, end): 
        """
        Calculate the shortest path from a starting node to an ending node in a sparse graph
        with potentially 10000s of nodes. Must run under 0.5 seconds and find the shortest distance between two nodes.
        
        Parameters:
        start: The starting node.
        end: The ending node.
        
        Returns:
        A tuple containing the total distance of the shortest path and a list of nodes representing that path.
        """
        # Your code here
        #return dist, path

        # Initialize the priority queue with the start vertex
        priority_queue = [(0, start)]  # (distance, vertex)
        distances = {vertex: float('inf') for vertex in self.graph}  # Set initial distances to infinity
        previous_nodes = {vertex: None for vertex in self.graph}  # To reconstruct the shortest path
        distances[start] = 0  # Distance to the start node is 0
        
        while priority_queue:
            current_distance, current_vertex = heapq.heappop(priority_queue)
            
            # If we've reached the target node, reconstruct the path
            if current_vertex == end:
                path = []
                while current_vertex:
                    path.insert(0, current_vertex)
                    current_vertex = previous_nodes[current_vertex]
                return current_distance, path
            
            # Skip any outdated entries in the priority queue
            if current_distance > distances[current_vertex]:
                continue
            
            # Explore neighbors
            for neighbor, weight in self.graph[current_vertex].items():
                distance = current_distance + weight
                
                # If a shorter path is found
                if distance < distances[neighbor]:
                    distances[neighbor] = distance
                    previous_nodes[neighbor] = current_vertex
                    heapq.heappush(priority_queue, (distance, neighbor))
        
        # If the end node is unreachable, return infinity and an empty path
        return float('inf'), []

    
    def tsp_small_graph(self, start_vertex): 
        """
        Solve the Travelling Salesman Problem for a small (~10 node) complete graph starting from a specified node.
        Required to find the optimal tour. Expect graphs with at most 10 nodes. Must run under 1 second.
        
        Parameters:
        start: The starting node.
        
        Returns:
        A tuple containing the total distance of the tour and a list of nodes representing the tour path.
        """
        # Your code here
        #return dist, path

        vertices = list(self.graph.keys())
        n = len(vertices)
    
        if n > 10:
            raise ValueError("Graph size exceeds limit for tsp_small_graph function.")
        
        if start_vertex not in vertices:
            raise ValueError("Starting vertex must be part of the graph.")
        
        # Mapping of each vertex to its index
        vertex_to_index = {vertex: i for i, vertex in enumerate(vertices)}
        start_index = vertex_to_index[start_vertex]
    
        # Precompute distances between every pair of vertices
        def distance(u, v):
            return self._get_edge_weight(u, v)
    
        # Dynamic programming table: dp[mask][i] is the shortest path to visit the subset of vertices
        # represented by 'mask', ending at vertex i
        dp = [[float('inf')] * n for _ in range(1 << n)]
        dp[1 << start_index][start_index] = 0  # Distance to start vertex is zero
    
        # Populate the dp table
        for mask in range(1 << n):
            for u in range(n):
                if not (mask & (1 << u)):  # If u is not in the subset
                    continue
                for v in range(n):
                    if mask & (1 << v):  # If v is already in the subset, skip
                        continue
                    # Update the dp table
                    dp[mask | (1 << v)][v] = min(
                        dp[mask | (1 << v)][v],
                        dp[mask][u] + distance(vertices[u], vertices[v])
                    )
    
        # Reconstruct the optimal tour from the dp table
        min_tour_distance = float('inf')
        last_vertex = -1
        final_mask = (1 << n) - 1
    
        # Find the minimum cost to return to the starting vertex
        for u in range(n):
            if u == start_index:
                continue
            tour_distance = dp[final_mask][u] + distance(vertices[u], start_vertex)
            if tour_distance < min_tour_distance:
                min_tour_distance = tour_distance
                last_vertex = u
    
        # Reconstruct the path
        optimal_tour = [start_vertex]
        mask = final_mask
        current_vertex = last_vertex
    
        while current_vertex != start_index:
            optimal_tour.append(vertices[current_vertex])
            prev_vertex = -1
            for u in range(n):
                if mask & (1 << u) and dp[mask][current_vertex] == dp[mask ^ (1 << current_vertex)][u] + distance(vertices[u], vertices[current_vertex]):
                    prev_vertex = u
                    break
            mask ^= (1 << current_vertex)
            current_vertex = prev_vertex
        
        optimal_tour.append(start_vertex)  # Complete the tour
    
        return min_tour_distance, optimal_tour[::-1]  # Return the optimal tour
    
    def tsp_large_graph(self, start): 
        """
        Solve the Travelling Salesman Problem for a large (~1000 node) complete graph starting from a specified node.
        No requirement to find the optimal tour. Must run under 0.5 second with a "pretty good" solution.
        
        Parameters:
        start: The starting node.
        
        Returns:
        A tuple containing the total distance of the tour and a list of nodes representing the tour path.
        """
        # Your code here
        # return dist, path

        # Initialize the set of unvisited nodes
        unvisited = set(self.graph.keys())
        
        # Ensure the starting vertex is part of the graph
        if start not in unvisited:
            raise ValueError("Starting vertex must be part of the graph.")
        
        # Initialize the tour
        tour = [start]
        unvisited.remove(start)
        
        current_vertex = start
        total_distance = 0
        
        # Helper function to get the weight between two vertices
        def distance(u, v):
            return self._get_edge_weight(u, v)
        
        # While there are still unvisited nodes
        while unvisited:
            # Find the nearest unvisited neighbor
            next_vertex = min(unvisited, key=lambda vertex: distance(current_vertex, vertex))
            
            # Add the nearest neighbor to the tour
            total_distance += distance(current_vertex, next_vertex)
            tour.append(next_vertex)
            unvisited.remove(next_vertex)
            
            # Move to the next vertex
            current_vertex = next_vertex
        
        # Return to the start vertex to complete the tour
        total_distance += distance(current_vertex, start)
        tour.append(start)
        
        return total_distance, tour
    
    def tsp_medium_graph(self, start): 
        """
        Solve the Travelling Salesman Problem for a medium (~300 node) complete graph starting from a specified node.
        Expected to perform better than tsp_large_graph. Must run under 1.5 seconds.
        
        Parameters:
        start: The starting node.
        
        Returns:
        A tuple containing the total distance of the tour and a list of nodes representing the tour path.
        """
        
        # Your code here
        # return dist, path

        # Initialize the set of unvisited nodes
        start_time = time.time()
        vertices = list(self.graph.keys())
        n = len(vertices)
        
        if start not in vertices:
            raise ValueError("Starting vertex must be part of the graph.")
        
        # Precompute distances between all pairs and store in a dictionary
        dist_matrix = {}
        def distance(u, v):
            if (u, v) not in dist_matrix:
                dist_matrix[(u, v)] = self._get_edge_weight(u, v)
            return dist_matrix[(u, v)]
        
        # Nearest Neighbor heuristic to generate an initial solution
        def nearest_neighbor(start):
            visited = {start}
            current_vertex = start
            tour = [start]
            total_distance = 0
    
            for _ in range(n - 1):
                next_vertex = min(
                    (v for v in vertices if v not in visited),
                    key=lambda v: distance(current_vertex, v)
                )
                total_distance += distance(current_vertex, next_vertex)
                tour.append(next_vertex)
                visited.add(next_vertex)
                current_vertex = next_vertex
    
            # Return to the starting point
            total_distance += distance(current_vertex, start)
            tour.append(start)
    
            return total_distance, tour
    
        # Optimized 2-opt to improve the initial solution
        def two_opt(tour, max_iterations=500, improvement_threshold=0.001):
            def calculate_total_distance(tour):
                return sum(distance(tour[i], tour[i + 1]) for i in range(len(tour) - 1))
    
            best_tour = tour
            best_distance = calculate_total_distance(best_tour)
            iteration = 0
            improved = True
    
            while iteration < max_iterations and improved:
                iteration += 1
                improved = False
                for i in range(1, n - 1):
                    for j in range(i + 1, n):
                        if j - i == 1:
                            continue
                        # Check if the new tour is shorter
                        if distance(best_tour[i - 1], best_tour[i]) + distance(best_tour[j], best_tour[j + 1]) > \
                           distance(best_tour[i - 1], best_tour[j]) + distance(best_tour[i], best_tour[j + 1]):
                            # Reverse segment tour[i:j]
                            best_tour[i:j] = best_tour[i:j][::-1]
                            best_distance = calculate_total_distance(best_tour)
                            improved = True
                            break  # Early termination if any improvement is found
                
                # Early stop if we are not getting significant improvements
                if (best_distance - initial_distance) / best_distance < improvement_threshold:
                    break
    
            return best_distance, best_tour
    
        # Step 1: Generate an initial solution using nearest neighbor
        initial_distance, initial_tour = nearest_neighbor(start)
        
        # Ensure we are still within time limit before applying 2-opt
        if time.time() - start_time > 1.2:
            return initial_distance, initial_tour
    
        # Step 2: Apply optimized 2-opt improvement
        optimized_distance, optimized_tour = two_opt(initial_tour)
    
        return optimized_distance, optimized_tour
        

## 3 - Exercises and Development Steps

### 3.1 - Exercises
Here are four exercises you'll complete in this assignment. Each one corresponds with a method that is already included in `Graph_Advanced`.

#### Exercise 1: `shortest_path` (Sparse Graphs, 10,000 nodes, Optimal Solution, 0.5 seconds)

In this challenge you will be given a large graph with tens of thousands of nodes. The graph is "sparse", however, meaning that each vertex may only have edges leading to a few other vertices. You will need to develop an algorithm to find the shortest path between two vertices in this graph. The solution will be the length of the path, and the list of vertices along that path. This is a classic computer science problem, and there are a few standard algorithms your LLM is likely to suggest. To ensure your code is efficient, it must execute in less than `0.5` seconds. 

#### Exercise 2: `tsp_small_graph` (Complete Graphs, 10 nodes, Optimal Solution, 1 second)

The Traveling Salesman Problem asks you to find the shortest path through a graph that visits all vertices and returns to the start, also called a "tour". This problem famously is computationally intensive for large graphs, making it essentially impossible to find the absolute best solution. For smaller graphs, however, a brute force approach is possible, and that's the first version of the Traveling Salesman Problem you'll tackle.

Write a method that, given a small graph of about 10 nodes, finds the shortest tour of the graph that starts and ends at node 0. Unlike in the first exercise, the graphs here are "complete", meaning there is an edge from each node to each other node. The method should return the length of the tour and a list of the nodes visited. **The tour found must be the absolute shortest through the graph.** Your solution must also be efficient, completing the task in under `1` second.

#### Exercise 3: `tsp_large_graph` (Complete Graphs, 1,000 nodes, "Pretty Good" Solution, 0.5 seconds)

In this exercise you again tackle the Traveling Salesman Problem, but for much larger graphs of about 1,000 nodes. Once again the graph is complete, with an edge between every pair of nodes. In graphs this size, a brute force approach is now computationally infeasible, so the tour length requirement on this method has been loosened substantially. You now must simply find a "pretty good" tour through the graph using some kind of heuristic approach. There are several commonly-used heuristics, and almost all of them (with the exception perhaps of randomly generating a tour) will produce tours that are short enough to pass the tests on this method. While your solution no longer needs to be optimal, your code should run quickly, in less than `0.5` seconds. Have your LLM focus on speed rather than tour length, and you should find an algorithm that works!

#### Exercise 4: `tsp_medium_graph` (Complete Graphs, 300 nodes, Near-Optimal Solution, 1.5 seconds)

In this last version of the Traveling Salesman Problem, you will need to work with an LLM to develop a solution that improves upon the algorithm you used in Exercise 3. Now you will be given complete graphs of about 300 nodes. The time requirement has been relaxed to `1.5` seconds, giving your algorithm 3 times as long to run. The tour length requirements, however, have been tightened, meaning you'll need to find tours that are much closer to the theoretical optimum. This likely means that the heuristic you used in Exercise 3 will no longer produce short enough tours. Brainstorm with your LLM new heuristics you could implement that take advantage of the longer runtime you're allowed and generate relatively shorter tours.

### 3.2 - Recommended Development Steps

The goal of this project is to develop your solutions alongside an LLM. Here's some steps to try:

- **Understand the Problem**: Discuss graph theory problems like the Travelling Salesman Problem (TSP) and shortest path finding with the LLM. Use it to clarify concepts and get examples, enhancing your foundational knowledge.

- **Analyze the `Graph` Class**: Analyze the provided Graph class functions and structure by consulting the LLM. Its explanations will help you understand how to effectively utilize and extend the class.

- **Brainstorm Solutions**: For each exercise and method (`shortest_path`, `tsp_small_graph`, `tsp_large_graph`, `tsp_medium_graph`), brainstorm with the LLM potential solutions. Share the time constraints and whether the solutions need to be optimal and see which algorithms the LLM recommends.

- **Implement Solutions**: Have the LLM generate methods that implement the solutions you brainstorm. Ensure the LLM understands the structure of the `Graph` and `Graph_Advanced` class so that the generated code will run as expected.

5. **Debug Errors**: Use the LLM to strategize and review your testing approach, especially for tests that yield unexpected results. It can offer debugging and optimization advice to improve your solutions.

Of course the LLM Best Practices of "Be specific", "Assign a role", "Request an expert opinion", "Give feedback" should serve you well throughout!

## 4 - Playground

Use the space below to experiment with and test your methods. The `generate_graph` function can be used to generate graphs with different properties.

To measure the execution time of your code, you can use the `%%timeit` magic method, an example of which appears at the bottom of this section. **Remember, `%%timeit` should be placed at the beginning of a code cell**, even before any comments marked by `#`.

You're encouraged to create as many new cells as needed for testing, but keep in mind that only the code within the `Graph_Advanced` class will be considered during grading.

### 4.1 - The `generate_graph` function

The function below will generate graphs and you may find it useful in experimenting with solutions or testing your code. This function is also the one used to generate the graphs used in the unit tests. If your algorithm fails a test case, you will be given the call to this function with the appropriate arguments needed to replicate the graph that caused your algorithm to fail. Additionally, the reason for the failure will be provided, whether it was due to exceeding the time limit or not achieving an optimal or near-optimal distance.

In [70]:
def generate_graph(nodes, edges=None, complete=False, weight_bounds=(1,600), seed=None):
    """
    Generates a graph with specified parameters, allowing for both complete and incomplete graphs.
    
    This function creates a graph with a specified number of nodes and edges, with options for creating a complete graph, and for specifying the weight bounds of the edges. It uses the Graph_Advanced class to create and manipulate the graph.

    Parameters:
    - nodes (int): The number of nodes in the graph. Must be a positive integer.
    - edges (int, optional): The number of edges to add for each node in the graph. This parameter is ignored if `complete` is set to True. Defaults to None.
    - complete (bool, optional): If set to True, generates a complete graph where every pair of distinct vertices is connected by a unique edge. Defaults to False.
    - weight_bounds (tuple, optional): A tuple specifying the lower and upper bounds (inclusive) for the random weights of the edges. Defaults to (1, 600).
    - seed (int, optional): A seed for the random number generator to ensure reproducibility. Defaults to None.

    Raises:
    - ValueError: If `edges` is not None and `complete` is set to True, since a complete graph does not require specifying the number of edges.

    Returns:
    - Graph_Advanced: An instance of the Graph_Advanced class representing the generated graph, with vertices labeled as integers starting from 0.

    Examples:
    - Generating a complete graph with 5 nodes:
        generate_graph(5, complete=True)
    
    - Generating an incomplete graph with 5 nodes and 2 edges per node:
        generate_graph(5, edges=2)
    
    Note:
    - The function assumes the existence of a Graph_Advanced class with methods for adding vertices (`add_vertex`) and edges (`add_edge`), as well as a method for getting adjacent vertices (`get_adjacent_vertices`).
    """
    random.seed(seed)
    graph = Graph_Advanced()
    if edges is not None and complete:
        raise ValueError("edges must be None if complete is set to True")
    if not complete and edges > nodes:
        raise ValueError("number of edges must be less than number of nodes")
    

    for i in range(nodes):
        graph.add_vertex(i)
    if complete:
        for i in range(nodes):
            for j in range(i+1,nodes):
                weight = random.randint(weight_bounds[0], weight_bounds[1])
                graph.add_edge(i,j,weight)
    else:
        for i in range(nodes):
            for _ in range(edges):
                j = random.randint(0, nodes - 1)
                while (j == i or j in graph.get_adjacent_vertices(i)) and len(graph.get_adjacent_vertices(i)) < nodes - 1:  # Ensure the edge is not a loop or a duplicate
                    j = random.randint(0, nodes - 1)
                weight = random.randint(weight_bounds[0], weight_bounds[1])
                if len(graph.graph[i]) < edges and len(graph.graph[j]) < edges:
                    graph.add_edge(i, j, weight)
    return graph

In [71]:
## Example function and use of the %%timeit magic method
def foo(n):
    i = 0
    for i in range(10000):
        for j in range(n):
            i += j
    return i

In [72]:
print(foo(10))

10044


In [73]:
%%timeit
foo(10)

4.97 ms ± 167 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Feel free to add as many cells as you want! You can do so by clicking in the `+` button in the left upper corner of this notebook.

## 5 - Test Your Solutions

The section below will run the methods in your `Graph_Advanced` class against a set of unit tests. If you see the message "All tests passed!" for every unittest, your assignment is likely ready for submission. If not, you'll be able to see which cases your code didn't pass and understand why. 

**Note, the unittests here cover only a few example cases due to the constraints of this environment. The grading process will include more extensive testing.** It's possible to pass all the tests here but still fail some during the autograding. However, if that happens, don't worry! You will receive feedback on which tests failed, which will allow you to adjust your solutions and resubmit the assignment. You're welcome to submit your assignment as many times as needed.

Here's a few tips to help you debug your code.

* Share the code of the `Graph` and `Graph_Advanced` classes with your LLM so it has that context. Make sure your code maintains the structure of the four methods, in particular the method names and their parameter and return types.
* Avoid importing new libraries besides those imported in Section 1. If you need additional libraries, only import them in the same cell that contains the `Graph_Advanced` class.
* Determine if the problem is with the accuray of your results or your code's runtime. Provide feedback to the LLM on reasons your solution is not passing and consider different algorithms / heuristics if you think the approach you're pursuing won't work.
* Save your work before submitting!
* Failing test because of long runtimes?** Ensure you shut down any other live kernels (if you have another notebook open within this environment), as they may affect the execution time of your functions. It's of course also possible that your code just needs to run more efficiently!

### 5.1 - Test Exercise 1 (`shortest_path`)

Run the code below to test the `shortest_path` method on sparsely connected graphs with 10,000 nodes. The requirements for passing this exercise are:

- The algorithm must complete its run in under `0.5` second for each graph.
- It must accurately find the shortest path.

In [74]:
unittests.test_shortest_path(Graph_Advanced)

 All tests passed!


### 5.2 - Test Exercise 2 (`tsp_small_graph`)

Run the code below to test the `tsp_small_graph` on complete (fully connected) graphs with 10 nodes. The requirements for passing this exercise are:

- The algorithm must complete its run in under `1` second for each graph.
- It must fund the optimal solution, starting at node 0.

In [ ]:
unittests.test_tsp_small_graph(Graph_Advanced)

### 5.3 - Test Exercise 3 (`tsp_large_graph`)

Run the code below to test the `tsp_large_graph` on complete (fully connected) graphs with 1000 nodes. The requirements for passing this exercise are:

- The algorithm must complete its run in under `0.5` second for each graph.
- It must find the good solution (less than a specified value, depending on the graph). 

In [ ]:
unittests.test_tsp_large_graph(Graph_Advanced)

### 5.4 - Test Exercise 4 (`tsp_medium_graph`)

Run the code below to test the `tsp_medium_graph` on complete (fully connected) graphs with 300 nodes. The requirements for passing this exercise are:

- The algorithm must complete its run in under `1.5` seconds for each graph.
- It must find the good solution (less than a specified value, depending on the graph). 

In [ ]:
unittests.test_tsp_medium_graph(Graph_Advanced)

Once you feel good about your submission, **save your work and submit!** 

**Congratulations on completing the first major assignment in this specialization!**